## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose start date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import importlib
import get_started_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
#import read_vars_for_separate_models_for_dependent_vars
import read_a_combination_of_variables
import yaml
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

In [2]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/started_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

# read the yaml file with data set parameters #
# data_set_name = input('Data set name: ')
data_set_name = 'CAN_CANCEL_spots_wo_CB_cancellation_requested'
with open(r'./parameters/data_sets.yaml') as file:
    data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)
    
spots_set = data_sets_parameters[data_set_name]['spots_set']
with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

if with_wo_CB == 'with_CB':
    with_wo_CB_boolean = True
else:
    with_wo_CB_boolean = False

if event_date_full_name == 'cancellation_requested':
    event_date = 'canc_req'
elif event_date_full_name == 'cancellation_confirmed':
    event_date = 'canc_conf'

In [3]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']

In [4]:
##### Model 0: all variables - prepare data for the model ####
cols_to_use = read_a_combination_of_variables.\
main(model_number=0, dir_name='combinations_of_variables_that_are_not_dependent/')
#### get behavioural variables ####
(variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
cols = base_cols + variables_to_use_for_the_model + \
did_something_before_and_didnt_last_month_vars + did_something_last_month_vars


#### get data for the model ###
(data, base_df, df_timeline_all_vars) = \
prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                            C = 100)
    
model_numbers = \
read_a_combination_of_variables.get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [5]:
for model_number in model_numbers[1:]:
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name='combinations_of_variables_that_are_not_dependent/')
    display(Markdown("# Model "+ str(model_number) + ": " + model_name))

    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
    get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = variables_to_use_for_the_model + \
    did_something_before_and_didnt_last_month_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    skip_vars

    ctv = fit_tv_cox_models.fit_the_models_and_print_summaries(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    ### save coefs and p values for p < 0.2 ###
    fit_tv_cox_models.save_results(df = ctv.summary.reset_index(), date_of_analysis = date_of_analysis, \
                                   data_dir = churn_based_on_behaviour_dir, dir_name='exports/coefficients_and_pvalues',\
                 results_name = 'coef_and_pvalues', spots_set = spots_set,\
                 with_wo_CB = with_wo_CB, event_date_type = event_date_full_name, p_limit=0.2, model_number=model_number,\
                                  model_type = model_type)

# Model 1: posts_on_facebook

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 16683 periods, 1365 subjects, 164 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 1365
 number of periods = 16683
  number of events = 164
partial log-likelihood = -943.03
  time fit was run = 2021-12-21 10:02:12 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Annual Prepay                                        0.56       1.75       0.18             0.21             0.91                 1.24                 2.49
New.email.subscriber.signups.last.month.div10       -0.10       0.91       0.06            -0.22             0.03                 0.80                 1.03
Website.Views.last.month.total.log2                 -0.17       0.84       0.06            -0.29            -0.05                 0.75                 0.95
had_added_events_manually_edited_events_before_...   0.15       1.16       0.19            -0.23             0.53                 0.80                 1.70
had_added_events_manually_edited_events_last_3_...  -0.64       0.53       0.34            -1.31             0.03                 0.27                 1.03
had_catering_submissions_before_and_didnt_last_...   0.17       1.18       0.27            -0.35             0.69                 0.70                 1.99
had_catering_submissions_last_3_months              -0.48       0.62       0.26            -0.99             0.03                 0.37                 1.03
had_clicked_emails_before_and_didnt_last_3_months    0.81       2.26       0.21             0.39             1.23                 1.48                 3.44
had_clicked_emails_last_3_months                     0.09       1.09       0.25            -0.40             0.58                 0.67                 1.78
had_emails_sent_manually_scheduled_emails_sent_...   0.03       1.03       0.18            -0.33             0.39                 0.72                 1.47
had_emails_sent_manually_scheduled_emails_sent_...  -0.58       0.56       0.23            -1.03            -0.13                 0.36                 0.88
had_online_orders_before_and_didnt_last_month        0.76       2.13       0.32             0.12             1.39                 1.13                 4.01
had_online_orders_last_month                        -0.13       0.88       0.38            -0.87             0.61                 0.42                 1.83
had_posts_on_facebook_before_and_didnt_last_month    0.56       1.76       0.20             0.18             0.95                 1.20                 2.58
had_posts_on_facebook_last_month                     0.09       1.10       0.21            -0.32             0.51                 0.72                 1.67
had_qr_code_menu_scans_before_and_didnt_last_month  -0.43       0.65       0.35            -1.13             0.26                 0.32                 1.30
had_qr_code_menu_scans_last_month                   -0.39       0.67       0.23            -0.84             0.05                 0.43                 1.06
had_reservations_submissions_before_and_didnt_l...   0.18       1.19       0.28            -0.37             0.73                 0.69                 2.07
had_reservations_submissions_last_month             -0.70       0.50       0.40            -1.47             0.08                 0.23                 1.09
had_tickets_before_and_didnt_last_3_months          -0.86       0.42       0.19            -1.24            -0.49                 0.29                 0.62
had_tickets_last_3_months                           -0.25       0.78       0.22            -0.67             0.18                 0.51                 1.20
had_visited_special_page_before_and_didnt_last_...   0.03       1.03       0.20            -0.36            

# Model 2: changed_picture_or_text

...

<lifelines.CoxTimeVaryingFitter: fitted with 16683 periods, 1365 subjects, 164 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 1365
 number of periods = 16683
  number of events = 164
partial log-likelihood = -934.74
  time fit was run = 2021-12-21 11:02:45 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Annual Prepay                                        0.55       1.73       0.18             0.20             0.90                 1.22                 2.45
New.email.subscriber.signups.last.month.div10       -0.10       0.91       0.06            -0.22             0.02                 0.80                 1.02
Website.Views.last.month.total.log2                 -0.19       0.83       0.06            -0.30            -0.07                 0.74                 0.93
had_added_events_manually_edited_events_before_...   0.07       1.07       0.20            -0.31             0.45                 0.73                 1.58
had_added_events_manually_edited_events_last_3_...  -0.67       0.51       0.35            -1.35             0.00                 0.26                 1.00
had_catering_submissions_before_and_didnt_last_...   0.19       1.21       0.27            -0.33             0.72                 0.72                 2.04
had_catering_submissions_last_3_months              -0.43       0.65       0.26            -0.95             0.08                 0.39                 1.08
had_clicked_emails_before_and_didnt_last_3_months    0.77       2.17       0.22             0.35             1.20                 1.42                 3.32
had_clicked_emails_last_3_months                     0.10       1.10       0.25            -0.39             0.59                 0.68                 1.80
had_emails_sent_manually_scheduled_emails_sent_...   0.03       1.03       0.18            -0.33             0.38                 0.72                 1.47
had_emails_sent_manually_scheduled_emails_sent_...  -0.50       0.61       0.23            -0.95            -0.05                 0.39                 0.95
had_number_of_requests_for_new_text_fragment_be...   0.39       1.48       0.19             0.03             0.76                 1.03                 2.13
had_number_of_requests_for_new_text_fragment_la...   0.68       1.98       0.37            -0.04             1.40                 0.96                 4.05
had_online_orders_before_and_didnt_last_month        0.72       2.05       0.32             0.09             1.35                 1.09                 3.84
had_online_orders_last_month                        -0.13       0.88       0.38            -0.87             0.61                 0.42                 1.84
had_posts_liked_before_and_didnt_last_3_months       0.28       1.32       0.19            -0.10             0.66                 0.90                 1.93
had_posts_liked_last_3_months                       -0.54       0.58       0.29            -1.10             0.02                 0.33                 1.02
had_posts_on_facebook_before_and_didnt_last_month    0.46       1.58       0.21             0.05             0.86                 1.05                 2.37
had_posts_on_facebook_last_month                     0.04       1.04       0.22            -0.39             0.47                 0.68                 1.59
had_reservations_submissions_before_and_didnt_l...   0.20       1.22       0.28            -0.35             0.75                 0.70                 2.11
had_reservations_submissions_last_month             -0.67       0.51       0.40            -1.45             0.11                 0.23                 1.12
had_tickets_before_and_didnt_last_3_months          -0.83       0.43       0.19            -1.22            

# Model 33: New.email.subscribers.allinclusive.last.month.div10

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 16683 periods, 1365 subjects, 164 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 1365
 number of periods = 16683
  number of events = 164
partial log-likelihood = -934.43
  time fit was run = 2021-12-21 11:04:39 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Annual Prepay                                        0.56       1.76       0.18             0.21             0.92                 1.24                 2.50
Website.Views.last.month.total.log2                 -0.21       0.81       0.06            -0.32            -0.10                 0.73                 0.91
had_added_events_manually_edited_events_before_...   0.07       1.08       0.19            -0.31             0.45                 0.73                 1.58
had_added_events_manually_edited_events_last_3_...  -0.74       0.48       0.35            -1.41            -0.06                 0.24                 0.94
had_catering_submissions_before_and_didnt_last_...   0.18       1.20       0.27            -0.34             0.71                 0.71                 2.03
had_catering_submissions_last_3_months              -0.47       0.62       0.26            -0.98             0.04                 0.37                 1.04
had_clicked_emails_before_and_didnt_last_3_months    0.78       2.18       0.22             0.35             1.20                 1.42                 3.33
had_clicked_emails_last_3_months                     0.10       1.11       0.25            -0.39             0.59                 0.68                 1.81
had_emails_sent_manually_scheduled_emails_sent_...   0.03       1.04       0.18            -0.32             0.39                 0.72                 1.48
had_emails_sent_manually_scheduled_emails_sent_...  -0.48       0.62       0.23            -0.93            -0.03                 0.40                 0.97
had_number_of_requests_for_new_text_fragment_be...   0.40       1.49       0.19             0.04             0.76                 1.04                 2.15
had_number_of_requests_for_new_text_fragment_la...   0.67       1.95       0.37            -0.05             1.39                 0.95                 4.00
had_online_orders_before_and_didnt_last_month        0.77       2.16       0.32             0.13             1.40                 1.14                 4.07
had_online_orders_last_month                        -0.07       0.93       0.38            -0.81             0.67                 0.44                 1.96
had_posts_liked_before_and_didnt_last_3_months       0.30       1.34       0.19            -0.09             0.68                 0.92                 1.97
had_posts_liked_last_3_months                       -0.53       0.59       0.29            -1.09             0.04                 0.34                 1.04
had_posts_on_facebook_before_and_didnt_last_month    0.46       1.58       0.21             0.05             0.87                 1.05                 2.38
had_posts_on_facebook_last_month                     0.04       1.04       0.22            -0.39             0.47                 0.68                 1.59
had_qr_code_menu_scans_before_and_didnt_last_month  -0.40       0.67       0.36            -1.10             0.30                 0.33                 1.35
had_qr_code_menu_scans_last_month                   -0.37       0.69       0.23            -0.83             0.09                 0.44                 1.09
had_reservations_submissions_before_and_didnt_l...   0.24       1.27       0.28            -0.31             0.79                 0.73                 2.21
had_reservations_submissions_last_month             -0.63       0.53       0.40            -1.42            